# DataSens E1_v2 — 05_snapshot_and_readme

- Objectifs: manifest JSON, MAJ versioning, rappel logs
- Prérequis: 04_quality_checks
- Sorties: `data/raw/manifests/manifest_*.json` + `README_VERSIONNING.md`
- Guide: docs/GUIDE_TECHNIQUE_E1.md



> Notes:
> - Génère un manifest JSON (traçabilité: version, timestamp, sources).
> - Met à jour `README_VERSIONNING.md` pour garder l’historique.
> - À adapter selon les sources réellement activées (OWM, RSS, NewsAPI, GDELT…).



In [ ]:
# DataSens E1_v2 - 05_snapshot_and_readme
# Manifest + versioning + snapshot hint
import json
from datetime import UTC, datetime
from pathlib import Path

ROOT = Path.cwd()
RAW = ROOT / "data" / "raw"
manifest = {
  "notebook_version": "E1_v2",
  "created_utc": datetime.now(UTC).isoformat(),
  "sources": ["Kaggle CSV","OpenWeatherMap","RSS","Web Scraping","GDELT"],
}
RAW.joinpath("manifests").mkdir(parents=True, exist_ok=True)
path = RAW / "manifests" / f"manifest_{datetime.now(UTC).strftime('%Y%m%dT%H%M%SZ')}.json"
path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")
print("✅ Manifest:", path)

# Versioning file
VERSION_FILE = ROOT / "README_VERSIONNING.md"
with open(VERSION_FILE, "a", encoding="utf-8") as f:
    f.write(f"- **{datetime.now(UTC).strftime('%Y-%m-%d %H:%M:%S UTC')}** | `E1_V2_COMPLETE` | Découpage v2 en 5 notebooks\n")
print("📘 Versionning mis à jour:", VERSION_FILE)

